In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import WordNetLemmatizer

In [ ]:
text = """A Christmas tree that can receive text messages has been unveiled at London's Tate Britain art gallery.

The spruce has an antenna which can receive Bluetooth texts sent by visitors to the Tate. The messages will be "unwrapped" by sculptor Richard Wentworth, who is responsible for decorating the tree with broken plates and light bulbs. It is the 17th year that the gallery has invited an artist to dress their Christmas tree. Artists who have decorated the Tate tree in previous years include Tracey Emin in 2002.

The plain green Norway spruce is displayed in the gallery's foyer. Its light bulb adornments are dimmed, ordinary domestic ones joined together with string. The plates decorating the branches will be auctioned off for the children's charity ArtWorks. Wentworth worked as an assistant to sculptor Henry Moore in the late 1960s. His reputation as a sculptor grew in the 1980s, while he has been one of the most influential teachers during the last two decades. Wentworth is also known for his photography of mundane, everyday subjects such as a cigarette packet jammed under the wonky leg of a table. """

In [1]:
#lemma
def lemmatization_func(text:str) -> str:
    tokenizer = ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens] 
    lemma_word = []
    wordnet_lemmatizer = WordNetLemmatizer()
    for w in tokens:
        word1 = wordnet_lemmatizer.lemmatize(w, pos = "n")
        word2 = wordnet_lemmatizer.lemmatize(word1, pos = "v")
        word3 = wordnet_lemmatizer.lemmatize(word2, pos = ("a"))
        lemma_word.append(word3)
    preprocessed_text = ' '.join(lemma_word)

    return preprocessed_text

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stopWords = set(stopwords.words("english"))
lemma_text = lemmatization_func(text)
words = word_tokenize(text)

freqTable = dict()
for word in words:
  word = word.lower()
  if word in stopWords:
    continue
  if word in freqTable:
    freqTable[word] += 1
  else:
    freqTable[word] = 1

sentences = sent_tokenize(text)
sentenceValue = dict()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
for sentence in sentences:
  for word, freq in freqTable.items():
    if word in sentence.lower():
      if sentence in sentenceValue:
        sentenceValue[sentence] += freq
      else:
        sentenceValue[sentence] = freq

sumValues = 0
for sentence in sentenceValue:
  sumValues += sentenceValue[sentence]

average = int(sumValues / len(sentenceValue))

summary = ''
for sentence in sentences:
  if(sentence in sentenceValue) and (sentenceValue[sentence] > (1.2 * average)):
    summary += " " + sentence
print(summary)

 A Christmas tree that can receive text messages has been unveiled at London's Tate Britain art gallery. The messages will be "unwrapped" by sculptor Richard Wentworth, who is responsible for decorating the tree with broken plates and light bulbs.


In [ ]:
!pip install rouge
from rouge import Rouge
ROUGE = Rouge()

In [ ]:
reference="The messages will be 'unwrapped' by sculptor Richard Wentworth, who is responsible for decorating the tree with broken plates and light bulbs.A Christmas tree that can receive text messages has been unveiled at London's Tate Britain art gallery.It is the 17th year that the gallery has invited an artist to dress their Christmas tree.The spruce has an antenna which can receive Bluetooth texts sent by visitors to the Tate.His reputation as a sculptor grew in the 1980s, while he has been one of the most influential teachers during the last two decades."
candidate = summary


In [ ]:
# just see recall
ROUGE.get_scores(candidate, reference)

[{'rouge-1': {'f': 0.6666666621622086,
   'p': 0.972972972972973,
   'r': 0.5070422535211268},
  'rouge-2': {'f': 0.5384615343242604,
   'p': 0.9210526315789473,
   'r': 0.3804347826086957},
  'rouge-l': {'f': 0.6666666621622086,
   'p': 0.972972972972973,
   'r': 0.5070422535211268}}]